# Dask starter notebook

This notebook introduces the basics of using dask with delayed functions and bags.

You don't need to write any code to execute this notebook, just step through each cell.

In [1]:
import dask
import dask.bag as db

# We'll use sleep to simulate some real work
from time import sleep

In [2]:
# We'll define a function that sleeps for 1 second to simulate work,
# then returns the square of its input.

def square(x):
    sleep(1)
    return x*x

In [3]:
# How long does it take to run a sum-of-squares serially?
%time sum([square(z) for z in range(10)])

CPU times: user 930 µs, sys: 162 µs, total: 1.09 ms
Wall time: 10 s


285

In [4]:
# Make a delayed version of square
f = dask.delayed(square)

# y is now a delayed computation
y = f(5)

In [5]:
y

Delayed('square-76dbf083-227f-475a-b08b-bd60c9fc4d40')

In [6]:
# We can derive a delayed computation for our sum above
z = dask.delayed(sum)([f(x) for x in range(10)])

In [7]:
# How long does it take to run the delayed summation with dask?
%time z.compute()

CPU times: user 1.26 ms, sys: 2.05 ms, total: 3.31 ms
Wall time: 10 s


285

In [8]:
# How long does it take to run the delayed summation with dask if we force the number of workers using thread mode
%time z.compute(scheduler='threads', num_workers=5)

CPU times: user 1.73 ms, sys: 753 µs, total: 2.48 ms
Wall time: 2.01 s


285

In [ ]:
# How long does it take to run the delayed summation with dask if using process mode
%time z.compute(scheduler='processes', num_workers=5)

In [ ]:
# How long does it take to run the delayed summation with dask if we use synchronous mode
%time z.compute(scheduler='synchronous', num_workers=5)

## Bags

The code above uses only delayed functions to express parallel computation.
We can also use bags to represent our data, and have a slightly higher-level control over things.

In [3]:
# Make a bag out of the sequence [0, 1, 2, 3, 4]
# Give it 3 partitions
bag = db.from_sequence(range(5), npartitions=3)

# Map the square function over the bag
c = bag.map(square)

# Apply the sum function to the result
d = c.sum()

In [4]:
bag

dask.bag<from_sequence, npartitions=3>

In [ ]:
# Mapping a function over a bag produces a new bag with the same partition structure
c

In [ ]:
# We can visualize the computation as follows:
c.visualize()

In [ ]:
# Applying a sum() reduction gives us a single Item (not a bag)
d

In [ ]:
d.visualize()

In [ ]:
d.compute()